# last 4 layers as avg

embeddings_and_labels_last_4_layers_avg.pkl

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer # Assuming you're loading these like this elsewhere

# Custom collate_fn to handle the tokenized dataset
# The default collator works fine if you set_format("torch")
# But for clarity, we can define it.
def collate_fn(batch):
    # This function takes a list of individual dataset items (dictionaries)
    # and stacks their components into tensors suitable for a batch.
    # It ensures that input_ids, attention_mask are tensors if they aren't already
    input_ids = torch.stack([torch.tensor(item['input_ids']) if not isinstance(item['input_ids'], torch.Tensor) else item['input_ids'] for item in batch])
    attention_mask = torch.stack([torch.tensor(item['attention_mask']) if not isinstance(item['attention_mask'], torch.Tensor) else item['attention_mask'] for item in batch])
    labels = torch.tensor([item['labels'] for item in batch]) # Labels are usually single integers, so directly convert to tensor
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}


def get_bert_embeddings(model, tokenizer, dataset, batch_size=32):
    # 1. Setup and Device Placement
    model.eval() # Set model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_embeddings = [] # List to store extracted embeddings from all batches
    all_labels = []     # List to store corresponding labels from all batches


    # Creates an iterable over your dataset that yields batches of data
    data_loader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)

    # 3. Inference Loop (No Gradient Calculation)
    with torch.no_grad(): # Disable gradient calculation for inference
        for batch in data_loader: # Iterate over batches provided by the DataLoader
            # Move the batch tensors to the appropriate device (GPU/CPU)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # 4. BERT Model Forward Pass for Hidden States
            # Get model outputs. output_hidden_states=True is crucial to get all layer outputs.
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)

            # outputs.hidden_states is a tuple of 13 tensors for BERT-base:
            # - Element 0: Embedding layer output (before the first Transformer layer)
            # - Elements 1 to 12: Outputs of the 12 Transformer encoder layers
            # We want the last 4 layers, which represent the most abstract and contextualized representations.
            # So, indices -1, -2, -3, -4 correspond to the last four encoder layers.
            last_4_layers_hidden_states = outputs.hidden_states[-4:] # Tuple of 4 tensors, each (batch_size, seq_len, hidden_size)

            # 5. Aggregating Across Layers (Average Pooling instead of Max Pooling)
            # Stacks the 4 tensors along a new dimension (dimension 0).
            # Shape changes from (4 x (batch_size, seq_len, hidden_size)) to (4, batch_size, seq_len, hidden_size)
            stacked_hidden_states = torch.stack(last_4_layers_hidden_states)

            # --- *** MODIFIED LINE HERE: Changed from torch.max to torch.mean *** ---
            # Takes the mean value element-wise across the new 'layer' dimension (dim=0).
            # This means for each token and each hidden dimension, it calculates the average value
            # from its representation in the last 4 layers.
            # Resulting shape: (batch_size, seq_len, hidden_size)
            avg_pooled_layers = torch.mean(stacked_hidden_states, dim=0)
            # --- *** END MODIFIED LINE *** ---


            # 6. Aggregating Across Tokens (Attention Masking and Average Pooling)
            # The avg_pooled_layers still contain embeddings for padding tokens (0s).
            # We need to exclude these when averaging to get a meaningful sentence embedding.

            # Expands the attention mask (batch_size, seq_len) to match the hidden_size dimension.
            # Shape becomes (batch_size, seq_len, 1), then expanded to (batch_size, seq_len, hidden_size).
            # This allows element-wise multiplication with the embeddings.
            attention_mask_expanded = attention_mask.unsqueeze(-1).expand_as(avg_pooled_layers)

            # Multiplies the embeddings by the expanded attention mask.
            # This sets the embeddings of padding tokens to zero, effectively ignoring them.
            masked_embeddings = avg_pooled_layers * attention_mask_expanded

            # Sums the embeddings along the sequence length dimension (dim=1) for each sample in the batch.
            # This gives a single vector for each sequence representing the sum of its token embeddings.
            # Resulting shape: (batch_size, hidden_size)
            sum_embeddings = torch.sum(masked_embeddings, dim=1)

            # Counts the number of non-padding tokens in each sequence.
            # Summing the attention mask (which contains 1s for real tokens and 0s for padding)
            # gives the actual length of each original sequence.
            # unsqueeze(-1) adds a dimension for broadcasting during division.
            # Shape: (batch_size, 1)
            num_tokens = torch.sum(attention_mask, dim=1).unsqueeze(-1)

            # Prevents division by zero if an attention mask somehow ends up with zero tokens (unlikely for valid inputs).
            num_tokens = torch.clamp(num_tokens, min=1e-9)

            # Calculates the average embedding for each sequence by dividing the sum of embeddings
            # by the number of actual tokens. This is the final document-level embedding.
            # Resulting shape: (batch_size, hidden_size)
            sentence_embeddings = sum_embeddings / num_tokens

            # 7. Collect Results
            # Appends the numpy representation of the batch's sentence embeddings to a list.
            all_embeddings.append(sentence_embeddings.cpu().numpy())
            # Appends the numpy representation of the batch's labels to a list.
            all_labels.append(labels.cpu().numpy())

    # 8. Final Output
    # Stacks all collected batch embeddings vertically to form a single NumPy array.
    # Resulting shape: (total_samples, hidden_size)
    # Stacks all collected batch labels horizontally to form a single 1D NumPy array.
    # Resulting shape: (total_samples,)
    return np.vstack(all_embeddings), np.hstack(all_labels)


# --- How the function is called ---

# 9. Extracting Embeddings for Training Data
print("\nExtracting BERT embeddings for training data (avg pooling last 4 layers)...")
# Calls the function for your training dataset.
# `fine_tuned_bert_embeddings_extractor` is the BERT model backbone (AutoModel)
# that has been fine-tuned on your fake news data.
train_embeddings, train_labels = get_bert_embeddings(bert_model, tokenizer, tokenized_datasets['train'])
print(f"Train Embeddings shape: {train_embeddings.shape}") # Prints the shape of the resulting array (e.g., (N_train_samples, 768))
print(f"Train Labels shape: {train_labels.shape}")       # Prints the shape of the labels array (e.g., (N_train_samples,))

# 10. Extracting Embeddings for Test Data
print("\nExtracting BERT embeddings for test data (avg pooling last 4 layers)...")
# Calls the function similarly for your test dataset.
# The 'validation' key here should correspond to your test split (as per your DatasetDict)
test_embeddings, test_labels = get_bert_embeddings(bert_model, tokenizer, tokenized_datasets['validation'])
print(f"Test Embeddings shape: {test_embeddings.shape}") # Prints the shape (e.g., (N_test_samples, 768))
print(f"Test Labels shape: {test_labels.shape}")       # Prints the shape (e.g., (N_test_samples,))

# concatenated cls tokens of last 4 layers

embeddings_and_labels_concat_last_4_layers_cls.pkl

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer

# Assuming AutoModel and AutoTokenizer are loaded correctly, e.g.:
# model_name = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# bert_model = AutoModel.from_pretrained(model_name)

# Custom collate_fn to handle the tokenized dataset
def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(item['input_ids']) if not isinstance(item['input_ids'], torch.Tensor) else item['input_ids'] for item in batch])
    attention_mask = torch.stack([torch.tensor(item['attention_mask']) if not isinstance(item['attention_mask'], torch.Tensor) else item['attention_mask'] for item in batch])
    labels = torch.tensor([item['labels'] for item in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

def get_bert_embeddings_cls_concat(model, tokenizer, dataset, batch_size=32, num_last_layers=4):
    """
    Extracts sentence embeddings by concatenating the [CLS] tokens
    from the last N layers of the BERT model.
    """
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_embeddings = []
    all_labels = []

    data_loader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)

            # Get the hidden states of the specified last N layers
            # outputs.hidden_states is a tuple where each element is (batch_size, seq_len, hidden_size)
            # The indices -num_last_layers: will select the last N layers
            selected_layers_hidden_states = outputs.hidden_states[-num_last_layers:]

            # Extract the [CLS] token embedding (index 0) from each of these layers
            cls_embeddings_from_layers = [
                layer_output[:, 0, :] # Shape: (batch_size, hidden_size)
                for layer_output in selected_layers_hidden_states
            ]

            # Concatenate these [CLS] embeddings along the feature dimension
            # Resulting shape: (batch_size, num_last_layers * hidden_size)
            sentence_embeddings = torch.cat(cls_embeddings_from_layers, dim=-1)

            all_embeddings.append(sentence_embeddings.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    return np.vstack(all_embeddings), np.hstack(all_labels)


# --- Example Usage ---
# Make sure you have 'bert_model', 'tokenizer', and 'tokenized_datasets' loaded.
# For example:
# from transformers import AutoModel, AutoTokenizer
# from datasets import DatasetDict, Dataset
#
# # Load a pre-trained BERT model and tokenizer (or your fine-tuned one)
# model_name = "bert-base-uncased" # Or your fine-tuned model path/name
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# bert_model = AutoModel.from_pretrained(model_name) # Or AutoModelForSequenceClassification.from_pretrained(...)
#
# # Placeholder for your tokenized_datasets (replace with your actual data)
# # Ensure 'input_ids', 'attention_mask', and 'labels' are present
# tokenized_datasets = DatasetDict({
#     'train': Dataset.from_dict({
#         'input_ids': [[101, 2054, 2003, 102], [101, 2047, 102]],
#         'attention_mask': [[1, 1, 1, 1], [1, 1, 1]],
#         'labels': [0, 1]
#     }),
#     'validation': Dataset.from_dict({
#         'input_ids': [[101, 2054, 2003, 102], [101, 2047, 102]],
#         'attention_mask': [[1, 1, 1, 1], [1, 1, 1]],
#         'labels': [0, 1]
#     })
# })


print("\nExtracting BERT embeddings (concatenating [CLS] tokens of last 4 layers)...")
train_embeddings_cls_concat, train_labels_cls_concat = get_bert_embeddings_cls_concat(bert_model, tokenizer, tokenized_datasets['train'], num_last_layers=4)
print(f"Train Embeddings shape: {train_embeddings_cls_concat.shape}")

print("\nExtracting BERT embeddings (concatenating [CLS] tokens of last 2 layers)...")
test_embeddings_cls_concat_2_layers, test_labels_cls_concat_2_layers = get_bert_embeddings_cls_concat(bert_model, tokenizer, tokenized_datasets['validation'], num_last_layers=2)
print(f"Test Embeddings shape: {test_embeddings_cls_concat_2_layers.shape}")

# last 4 layers. Avg concatenated

embeddings_and_labels_concat_last_4_layers_avg.pkl

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer

# Custom collate_fn to handle the tokenized dataset (same as above)
def collate_fn(batch):
    input_ids = torch.stack([torch.tensor(item['input_ids']) if not isinstance(item['input_ids'], torch.Tensor) else item['input_ids'] for item in batch])
    attention_mask = torch.stack([torch.tensor(item['attention_mask']) if not isinstance(item['attention_mask'], torch.Tensor) else item['attention_mask'] for item in batch])
    labels = torch.tensor([item['labels'] for item in batch])
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}


def get_bert_embeddings_avg_concat(model, tokenizer, dataset, batch_size=32, num_last_layers=4):
    """
    Extracts sentence embeddings by concatenating the attention-masked average pooled
    token representations from the last N layers of the BERT model.
    """
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_embeddings = []
    all_labels = []

    data_loader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)

            selected_layers_hidden_states = outputs.hidden_states[-num_last_layers:]

            layer_pooled_embeddings = []
            for layer_output in selected_layers_hidden_states:
                # Apply attention mask and average pooling for each layer's output
                attention_mask_expanded = attention_mask.unsqueeze(-1).expand_as(layer_output)
                masked_embeddings = layer_output * attention_mask_expanded
                sum_embeddings = torch.sum(masked_embeddings, dim=1)
                num_tokens = torch.sum(attention_mask, dim=1).unsqueeze(-1)
                num_tokens = torch.clamp(num_tokens, min=1e-9)
                avg_pooled_layer = sum_embeddings / num_tokens # Shape: (batch_size, hidden_size)
                layer_pooled_embeddings.append(avg_pooled_layer)

            # Concatenate these average pooled embeddings along the feature dimension
            # Resulting shape: (batch_size, num_last_layers * hidden_size)
            sentence_embeddings = torch.cat(layer_pooled_embeddings, dim=-1)

            all_embeddings.append(sentence_embeddings.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

    return np.vstack(all_embeddings), np.hstack(all_labels)

# --- Example Usage ---
# Make sure you have 'bert_model', 'tokenizer', and 'tokenized_datasets' loaded.

print("\nExtracting BERT embeddings (concatenating average pooled last 4 layers)...")
train_embeddings_avg_concat, train_labels_avg_concat = get_bert_embeddings_avg_concat(bert_model, tokenizer, tokenized_datasets['train'], num_last_layers=4)
print(f"Train Embeddings shape: {train_embeddings_avg_concat.shape}")

print("\nExtracting BERT embeddings (concatenating average pooled last 2 layers)...")
test_embeddings_avg_concat_2_layers, test_labels_avg_concat_2_layers = get_bert_embeddings_avg_concat(bert_model, tokenizer, tokenized_datasets['validation'], num_last_layers=2)
print(f"Test Embeddings shape: {test_embeddings_avg_concat_2_layers.shape}")

# cls token of last layer

embeddings_and_labels_cls_last_layer.pkl

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer # Assuming you're loading these like this elsewhere

# Custom collate_fn to handle the tokenized dataset
# The default collator works fine if you set_format("torch")
# But for clarity, we can define it.
def collate_fn(batch):
    # This function takes a list of individual dataset items (dictionaries)
    # and stacks their components into tensors suitable for a batch.
    # It ensures that input_ids, attention_mask are tensors if they aren't already
    input_ids = torch.stack([torch.tensor(item['input_ids']) if not isinstance(item['input_ids'], torch.Tensor) else item['input_ids'] for item in batch])
    attention_mask = torch.stack([torch.tensor(item['attention_mask']) if not isinstance(item['attention_mask'], torch.Tensor) else item['attention_mask'] for item in batch])
    labels = torch.tensor([item['labels'] for item in batch]) # Labels are usually single integers, so directly convert to tensor
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}


def get_bert_embeddings(model, tokenizer, dataset, batch_size=32):
    # 1. Setup and Device Placement
    model.eval() # Set model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_embeddings = [] # List to store extracted embeddings from all batches
    all_labels = []     # List to store corresponding labels from all batches


    # Creates an iterable over your dataset that yields batches of data
    data_loader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)

    # 3. Inference Loop (No Gradient Calculation)
    with torch.no_grad(): # Disable gradient calculation for inference
        for batch in data_loader: # Iterate over batches provided by the DataLoader
            # Move the batch tensors to the appropriate device (GPU/CPU)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # 4. BERT Model Forward Pass for Hidden States
            # Get model outputs. output_hidden_states=True is crucial to get all layer outputs.
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)

            # --- *** MODIFIED SECTION: Extracting CLS Token Embedding *** ---
            # outputs.last_hidden_state is the output of the final BERT layer.
            # Its shape is (batch_size, sequence_length, hidden_size).
            # The [CLS] token is always the first token (index 0) in the sequence.
            # So, outputs.last_hidden_state[:, 0, :] extracts the embedding
            # for the [CLS] token for all samples in the current batch.
            sentence_embeddings = outputs.last_hidden_state[:, 0, :]
            # --- *** END MODIFIED SECTION *** ---

            # 7. Collect Results
            # Appends the numpy representation of the batch's sentence embeddings to a list.
            all_embeddings.append(sentence_embeddings.cpu().numpy())
            # Appends the numpy representation of the batch's labels to a list.
            all_labels.append(labels.cpu().numpy())

    # 8. Final Output
    # Stacks all collected batch embeddings vertically to form a single NumPy array.
    # Resulting shape: (total_samples, hidden_size)
    # Stacks all collected batch labels horizontally to form a single 1D NumPy array.
    # Resulting shape: (total_samples,)
    return np.vstack(all_embeddings), np.hstack(all_labels)

# --- How the function is called (remains the same) ---
# Assuming 'bert_model' and 'tokenizer' are already defined and loaded,
# and 'tokenized_datasets' is your dataset dictionary.

# Example placeholder for your model and tokenizer (ensure these are loaded as they were before)
# from transformers import BertForSequenceClassification # if you fine-tuned a classification head
# model_name = "bert-base-uncased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# bert_model = AutoModel.from_pretrained(model_name) # or BertForSequenceClassification if that's what you fine-tuned

# Make sure tokenized_datasets is defined, e.g.:
# from datasets import DatasetDict, Dataset
# tokenized_datasets = DatasetDict({
#     'train': Dataset.from_dict({'input_ids': ..., 'attention_mask': ..., 'labels': ...}),
#     'validation': Dataset.from_dict({'input_ids': ..., 'attention_mask': ..., 'labels': ...})
# })


# 9. Extracting Embeddings for Training Data
print("\nExtracting BERT embeddings for training data (using [CLS] token)...")
train_embeddings, train_labels = get_bert_embeddings(bert_model, tokenizer, tokenized_datasets['train'])
print(f"Train Embeddings shape: {train_embeddings.shape}")
print(f"Train Labels shape: {train_labels.shape}")

# 10. Extracting Embeddings for Test Data
print("\nExtracting BERT embeddings for test data (using [CLS] token)...")
test_embeddings, test_labels = get_bert_embeddings(bert_model, tokenizer, tokenized_datasets['validation'])
print(f"Test Embeddings shape: {test_embeddings.shape}")
print(f"Test Labels shape: {test_labels.shape}")